In [ ]:
from collections import namedtuple

import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as T
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Beta-Binomial Regression
[TODO]

In [ ]:
def get_beta_binomial_regression_model(y_obs, y_n, covars):
    if y_obs.shape[0] != y_n.shape[0]:
        raise ValueError('Number of samples and observations must match!')
        
    if y_obs.shape[0] != covars.shape[0]:
        raise ValueError('Number of instances of covariates and samples must match!')
    
    num_obs = y_obs.shape[0]
    num_covars = covars.shape[1]
    
    model = pm.Model()
    
    with model:
        nu  = pm.Normal('nu', mu=0., sd=1., shape=num_covars)
        pi = pm.math.logit(T.dot(covars, nu))
        gamma = pm.HalfNormal('gamma', sd=1.)
        
        alpha = pm.Deterministic(('alpha', (1 - pi)/gamma - 1/gamma) * pi ** 2)
        beta = pm.Deterministic('beta', alpha * (1/pi - 1))
        
        y = pm.BetaBinomial('y', alpha, beta, y_n)
        
    return model

In [ ]:
Simulation = namedtuple('Simulation', ['y', 'n', 'X', 'nu'])

def sim_data(num_obs, num_covars, mean_sample):
    X = np.random.randn(num_obs,num_covars)
    nu = np.random.randn(num_covars, 1)
    
    Z = np.dot(X, nu)[0]
    pi = np.exp(Z) / (1 + np.exp(Z))
    
    n = np.random.poisson(mean_sample, num_obs)
    
    y = np.random.binomial(n, pi)
    
    return Simulation(y, n, X, nu)

In [ ]:
simulation = sim_data(100, 8, 12)

In [ ]:
beta_binomial_regression = get_beta_binomial_regression_model(simulation.y, simulation.n, simulation.X)